# Initialization

In [1]:
import logging
import scipy
import sys
import sklearn.preprocessing
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
from surprise import Dataset, Reader
from surprise import SVD
from surprise import accuracy
from surprise import NormalPredictor
from scipy.sparse import csr_matrix
from implicit.als import AlternatingLeastSquares
from IPython.display import display
from sklearn.metrics.pairwise import cosine_similarity
from catboost import CatBoostClassifier, Pool

/home/mle-user/sprint_4/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'png'
%config InlineBackend.figure_format = 'retina'

# Загрузка данных

In [2]:
items = pd.read_parquet("items.parquet")
events = pd.read_parquet("events.parquet")

# Разбиение с учётом хронологии

Рекомендательные системы на практике работают с учётом хронологии. Поэтому поток событий для тренировки и валидации полезно делить на то, что уже случилось, и что ещё случится. Это позволяет проводить валидацию на тех же пользователях, на которых тренировались, но на их событиях в будущем.

# === Знакомство: "холодный" старт

In [3]:
# зададим точку разбиения
train_test_global_time_split_date = pd.to_datetime("2017-08-01").date()
train_test_global_time_split_idx = events["started_at"] < train_test_global_time_split_date

# Разбиваем события на train и test
events_train = events[train_test_global_time_split_idx]
events_test = events[~train_test_global_time_split_idx]

# количество уникальных пользователей в train и test
users_train = events_train["user_id"].drop_duplicates()
users_test = events_test["user_id"].drop_duplicates()

# количество пользователей, которые есть и в train, и в test
common_users = pd.Index(users_train).intersection(pd.Index(users_test))

In [6]:
# Идентифицируем холодных пользователей
cold_users = pd.Index(users_test).difference(pd.Index(users_train))

In [7]:
# Задаем дату начала для подсчета популярности
top_pop_start_date = pd.to_datetime("2015-01-01").date()

In [8]:
# Группируем события по item_id и вычисляем количество уникальных пользователей и средний рейтинг
item_popularity = events_train \
    .query("started_at >= @top_pop_start_date") \
    .groupby(["item_id"]).agg(users=("user_id", "nunique"), avg_rating=("rating", "mean")).reset_index()

# Нормализация пользователей и среднего рейтинга, требуется для их приведения к одному масштабу
scaler = MinMaxScaler()
item_popularity[["users_norm", "avg_rating_norm"]] = scaler.fit_transform(
    item_popularity[["users", "avg_rating"]]
)

# Вычисляем popularity_score, как скор популярности со штрафом за низкий рейтинг
item_popularity["popularity_score"] = (
    item_popularity["users_norm"] * item_popularity["avg_rating_norm"]
)

# Сортируем по убыванию popularity_score
item_popularity = item_popularity.sort_values(by="popularity_score", ascending=False).reset_index(drop=True)

# Выбираем первые 100 айтемов со средней оценкой avg_rating не меньше 4
top_k_pop_items = item_popularity.query("avg_rating >= 4").head(100)

# Получаем item_id первой книги в топ-100
top_item_id = top_k_pop_items.iloc[0]["item_id"]

# Определяем количество пользователей, оценивших эту книгу
num_users_rated_top_item = events_train.query("item_id == @top_item_id and started_at >= @top_pop_start_date")["user_id"].nunique()

In [9]:
# добавляем информацию о книгах
top_k_pop_items = top_k_pop_items.merge(
    items.set_index("item_id")[["author", "title", "genre_and_votes", "publication_year"]], on="item_id")

with pd.option_context('display.max_rows', 100):
    display(top_k_pop_items[["item_id", "author", "title", "publication_year", "users", "avg_rating", "popularity_score", "genre_and_votes"]])

,item_id,author,title,publication_year,users,avg_rating,popularity_score,genre_and_votes
0,18007564,Andy Weir,The Martian,2014,20207,4.321275,0.412333,"{'Science Fiction': 11966, 'Fiction': 8430}"
1,18143977,Anthony Doerr,All the Light We Cannot See,2014,19462,4.290669,0.393471,"{'Historical-Historical Fiction': 13679, 'Fict..."
2,3,"J.K. Rowling, Mary GrandPré",Harry Potter and the Sorcerer's Stone (Harry P...,1997,15139,4.706057,0.344702,"{'Fantasy': 59818, 'Fiction': 17918, 'Young Ad..."
3,16096824,Sarah J. Maas,A Court of Thorns and Roses (A Court of Thorns...,2015,16770,4.301014,0.340108,"{'Fantasy': 14326, 'Young Adult': 4662, 'Roman..."
4,15881,"J.K. Rowling, Mary GrandPré",Harry Potter and the Chamber of Secrets (Harry...,1999,13043,4.632447,0.291076,"{'Fantasy': 50130, 'Young Adult': 15202, 'Fict..."
5,38447,Margaret Atwood,The Handmaid's Tale,1998,14611,4.232770,0.290194,"{'Fiction': 15424, 'Classics': 9937, 'Science ..."
6,11235712,Marissa Meyer,"Cinder (The Lunar Chronicles, #1)",2012,14348,4.179189,0.280247,"{'Young Adult': 10539, 'Fantasy': 9237, 'Scien..."
7,17927395,Sarah J. Maas,A Court of Mist and Fury (A Court of Thorns an...,2016,12177,4.730640,0.279094,"{'Fantasy': 10186, 'Romance': 3346, 'Young Adu..."
8,5,"J.K. Rowling, Mary GrandPré",Harry Potter and the Prisoner of Azkaban (Harr...,2004,11890,4.770143,0.275401,"{'Fantasy': 49784, 'Young Adult': 15393, 'Fict..."
9,13206900,Marissa Meyer,"Winter (The Lunar Chronicles, #4)",2015,12291,4.534293,0.266881,"{'Fantasy': 4835, 'Young Adult': 4672, 'Scienc..."


In [10]:
# Фильтруем события холодных пользователей
cold_users_events_with_recs = \
    events_test[events_test["user_id"].isin(cold_users)] \
    .merge(top_k_pop_items[["item_id", "avg_rating"]], on="item_id", how="left")

# Определяем индексы событий без оценки avg_rating
cold_user_items_no_avg_rating_idx = cold_users_events_with_recs["avg_rating"].isnull()

# Фильтруем события, у которых есть значение avg_rating
cold_user_recs = cold_users_events_with_recs[~cold_user_items_no_avg_rating_idx] \
    [["user_id", "item_id", "rating", "avg_rating"]]

# Считаем долю событий, для которых нашли соответствующий avg_rating
fraction_with_recommendation = len(cold_user_recs) / len(cold_users_events_with_recs)

In [11]:
# Посчитаем метрики рекомендаций
rmse = mean_squared_error(cold_user_recs["rating"], cold_user_recs["avg_rating"], squared=False)
mae = mean_absolute_error(cold_user_recs["rating"], cold_user_recs["avg_rating"])

In [12]:
# выводим результаты
print(f"Количество уникальных пользователей в events_train: {len(users_train)}")
print(f"Количество уникальных пользователей в events_test: {len(users_test)}")
print(f"Количество уникальных пользователей, которые есть и в events_train, и в events_test: {len(common_users)}")
print(f"Количество холодных пользователей: {len(cold_users)}")
print(f"Топ-1 item_id: {top_item_id}")
print(f"Количество пользователей, оценивших книгу с item_id {top_item_id}: {num_users_rated_top_item}")
print(f"Доля событий «холодных» пользователей в events_test, для которых нашли рекомендации: {fraction_with_recommendation:.2f}")
print(round(rmse, 2), round(mae, 2))


Количество уникальных пользователей в events_train: 428220
Количество уникальных пользователей в events_test: 123223
Количество уникальных пользователей, которые есть и в events_train, и в events_test: 120858
Количество холодных пользователей: 2365
Топ-1 item_id: 18007564.0
Количество пользователей, оценивших книгу с item_id 18007564.0: 20207
Доля событий «холодных» пользователей в events_test, для которых нашли рекомендации: 0.20
0.78 0.62


In [13]:
# посчитаем покрытие холодных пользователей рекомендациями

cold_users_hit_ratio = cold_users_events_with_recs.groupby("user_id").agg(hits=("avg_rating", lambda x: (~x.isnull()).mean()))

print(f"Доля пользователей без релевантных рекомендаций: {(cold_users_hit_ratio == 0).mean().iat[0]:.2f}")
print(f"Среднее покрытие пользователей: {cold_users_hit_ratio[cold_users_hit_ratio != 0].mean().iat[0]:.2f}")

Доля пользователей без релевантных рекомендаций: 0.59
Среднее покрытие пользователей: 0.44


In [14]:
# Количество уникальных пользователей
num_unique_users = events['user_id'].nunique()

# Количество уникальных элементов (книг)
num_unique_items = events['item_id'].nunique()

# Количество ненулевых элементов (рейтинги)
num_non_zero_entries = events[['user_id', 'item_id', 'rating']].drop_duplicates().shape[0]

# Общее число возможных элементов в матрице
total_possible_entries = num_unique_users * num_unique_items

# Степень разреженности
sparsity = 1 - (num_non_zero_entries / total_possible_entries)
sparsity_percentage = sparsity * 100

print(f"Степень разреженности матрицы U-I: {sparsity:.4f} или {sparsity_percentage:.2f}%")

Степень разреженности матрицы U-I: 0.9993 или 99.93%


# === Знакомство: первые персональные рекомендации

In [15]:
# используем Reader из библиотеки surprise для преобразования событий (events)
# в формат, необходимый surprise
reader = Reader(rating_scale=(1, 5))
surprise_train_set = Dataset.load_from_df(events_train[['user_id', 'item_id', 'rating']], reader)
surprise_train_set = surprise_train_set.build_full_trainset()

# инициализируем модель
svd_model = SVD(n_factors=100, random_state=0)

# обучаем модель
svd_model.fit(surprise_train_set) 

In [16]:
#Используя обученную модель, получим рекомендации для тестовой выборки:
surprise_test_set = list(events_test[['user_id', 'item_id', 'rating']].itertuples(index=False))

# получаем рекомендации для тестовой выборки
svd_predictions = svd_model.test(surprise_test_set) 

In [ ]:
#Оценка рекомендаций
rmse = accuracy.rmse(svd_predictions)
mae = accuracy.mae(svd_predictions)
                     
print(rmse, mae)

In [17]:
# инициализируем состояние генератора, это необходимо для получения
# одной и той же последовательности случайных чисел, только в учебных целях
np.random.seed(0)

random_model = NormalPredictor()

random_model.fit(surprise_train_set)
random_predictions = random_model.test(surprise_test_set)

In [ ]:
#Оценка рекомендаций
rmse_rand = accuracy.rmse(random_predictions)
mae_rand = accuracy.mae(random_predictions)
                     
print(rmse_rand, mae_rand)

In [ ]:
# Вычислим разницу между MAE случайной модели и SVD модели
difference_mae = mae_rand - mae

# Вычислим процентный рост MAE для случайной модели относительно SVD модели
percentage_increase = (difference_mae / mae) * 100
percentage_increase_rounded = round(percentage_increase)

print(f"MAE для случайной модели выше MAE для SVD модели на {percentage_increase_rounded}%")

In [18]:
def get_recommendations_svd(user_id, all_items, events, model, include_seen=True, n=10):
    """ Возвращает n рекомендаций для user_id """
    # Получим список идентификаторов всех книг
    all_items = set(events['item_id'].unique())
    
    # Учитываем флаг, стоит ли уже прочитанные книги включать в рекомендации
    if include_seen:
        items_to_predict = list(all_items)
    else:
        # Получим список книг, которые пользователь уже прочитал ("видел")
        seen_items = set(events[events['user_id'] == user_id]['item_id'].unique())
        # Книги, которые пользователь ещё не читал
        # Только их и будем включать в рекомендации
        items_to_predict = list(all_items - seen_items)
    
    # Получаем скоры для списка книг, т. е. рекомендации
    predictions = [model.predict(user_id, item_id) for item_id in items_to_predict]
    
    # Сортируем рекомендации по убыванию скора и берём только n первых
    recommendations = sorted(predictions, key=lambda x: x.est, reverse=True)[:n]
    
    return pd.DataFrame([(pred.iid, pred.est) for pred in recommendations], columns=["item_id", "score"])

In [ ]:
get_recommendations_svd(1296647, items, events_test, svd_model) 

In [19]:
# выберем произвольного пользователя из тренировочной выборки ("прошлого")
user_id = events_train['user_id'].sample().iat[0]

print(f"user_id: {user_id}")

print("История (последние события, recent)")
user_history = (
    events_train
    .query("user_id == @user_id")
    .merge(items.set_index("item_id")[["author", "title", "genre_and_votes"]], on="item_id")
)


user_id: 1087036
История (последние события, recent)


In [20]:
user_history_to_print = user_history[["author", "title", "started_at", "is_reviewed", "rating", "genre_and_votes"]].tail(10)
display(user_history_to_print)

print("Рекомендации")
user_recommendations = get_recommendations_svd(user_id, items, events_train, svd_model)
user_recommendations = user_recommendations.merge(items[["item_id", "author", "title", "genre_and_votes"]], on="item_id")
display(user_recommendations)

,author,title,started_at,is_reviewed,rating,genre_and_votes
68,Veronica Roth,"Divergent (Divergent, #1)",2014-06-02,False,4,"{'Young Adult': 20260, 'Science Fiction-Dystop..."
69,"Gillian Flynn, В. Русанов",Gone Girl,2014-05-27,True,5,"{'Fiction': 11773, 'Mystery': 9965, 'Thriller'..."
70,Kathy Reichs,"Death du Jour (Temperance Brennan, #2)",2014-05-24,False,4,"{'Mystery': 1206, 'Mystery-Crime': 579, 'Ficti..."
71,Chelsea Cain,"Heartsick (Archie Sheridan & Gretchen Lowell, #1)",2014-05-22,False,5,"{'Mystery': 832, 'Thriller': 653, 'Fiction': 4..."
72,"Jussi Adler-Olsen, Lisa Hartford","The Keeper of Lost Causes (Department Q, #1)",2014-05-30,False,3,"{'Mystery': 1225, 'Mystery-Crime': 627, 'Ficti..."
73,Gillian Flynn,Dark Places,2014-05-17,False,4,"{'Mystery': 4534, 'Fiction': 4055, 'Thriller':..."
74,Audrey Niffenegger,Her Fearful Symmetry,2014-05-05,False,2,"{'Fiction': 1984, 'Fantasy': 674, 'Fantasy-Par..."
75,Kathy Reichs,"Déjà Dead (Temperance Brennan, #1)",2014-05-13,False,4,"{'Mystery': 2141, 'Fiction': 904, 'Mystery-Cri..."
76,Carolyn Parkhurst,The Dogs of Babel,2014-05-09,False,5,"{'Fiction': 522, 'Mystery': 102, 'Animals': 77..."
77,George R.R. Martin,"A Dance with Dragons (A Song of Ice and Fire, #5)",2014-05-04,False,5,"{'Fantasy': 22247, 'Fiction': 4512, 'Fantasy-E..."


Рекомендации


,item_id,score,author,title,genre_and_votes
0,2199,5,Doris Kearns Goodwin,Team of Rivals: The Political Genius of Abraha...,"{'History': 4174, 'Nonfiction': 2127, 'Biograp..."
1,16255632,5,"David Gaider, Ben Gelinas, Mike Laidlaw, Dave ...",Dragon Age: The World of Thedas Volume 1,"{'Fantasy': 134, 'Games-Video Games': 28, 'Art..."
2,2363958,5,João Guimarães Rosa,Grande Sertão: Veredas,"{'Fiction': 85, 'Classics': 69, 'Cultural-Braz..."
3,22552026,5,Jason Reynolds,Long Way Down,"{'Young Adult': 1871, 'Poetry': 1737, 'Contemp..."
4,29237211,5,"Brian K. Vaughan, Fiona Staples","Saga, Vol. 7 (Saga, #7)","{'Sequential Art-Graphic Novels': 2539, 'Seque..."
5,9516,5,"Marjane Satrapi, Mattias Ripa",Persepolis: The Story of a Childhood (Persepol...,"{'Sequential Art-Graphic Novels': 4726, 'Nonfi..."
6,23602722,5,Kanata Konami,"Chi's Sweet Home, Volume 12","{'Sequential Art-Manga': 90, 'Sequential Art-G..."
7,147915,5,George R.R. Martin,A Storm of Swords: Blood and Gold (A Song of I...,"{'Fantasy': 3256, 'Fiction': 557, 'Fantasy-Epi..."
8,22037424,5,"J.K. Rowling, Jonny Duddle, Tomislav Tomić",Harry Potter and the Prisoner of Azkaban (Harr...,"{'Fantasy': 49994, 'Young Adult': 15433, 'Fict..."
9,24812,5,Bill Watterson,The Complete Calvin and Hobbes,"{'Sequential Art-Comics': 867, 'Humor': 378, '..."


In [21]:
# выберем произвольного пользователя из тренировочной выборки ("прошлого")
user_id = events_train['user_id'].sample().iat[0]

print(f"user_id: {user_id}")

print("История (последние события, recent)")
user_history = (
    events_train
    .query("user_id == @user_id")
    .merge(items.set_index("item_id")[["author", "title", "genre_and_votes"]], on="item_id")
)
user_history_to_print = user_history[["author", "title", "started_at", "is_reviewed", "rating", "genre_and_votes"]].tail(10)
display(user_history_to_print)

print("Рекомендации")
user_recommendations = get_recommendations_svd(user_id, items, events_train, svd_model)
user_recommendations = user_recommendations.merge(items[["item_id", "author", "title", "genre_and_votes"]], on="item_id")
display(user_recommendations)

user_id: 1256676
История (последние события, recent)


,author,title,started_at,is_reviewed,rating,genre_and_votes
40,Nick Cave,The Death of Bunny Munro,2013-02-22,False,5,"{'Fiction': 340, 'Thriller': 55, 'Contemporary..."
41,David Byrne,How Music Works,2013-02-04,False,4,"{'Music': 1012, 'Nonfiction': 554, 'History': ..."
42,"Haruki Murakami, Alfred Birnbaum, Philip Gabriel",Underground: The Tokyo Gas Attack and the Japa...,2012-08-08,False,4,"{'Nonfiction': 785, 'Cultural-Japan': 336, 'Hi..."
43,Jennifer Egan,A Visit from the Goon Squad,2012-07-10,False,4,"{'Fiction': 3761, 'Contemporary': 543, 'Music'..."
44,Oscar Wilde,The Happy Prince,2012-07-08,False,4,"{'Classics': 552, 'Short Stories': 273, 'Ficti..."
45,"Roald Dahl, Quentin Blake",The Twits,2012-05-12,False,3,"{'Childrens': 1359, 'Fiction': 939, 'Fantasy':..."
46,Kazuo Ishiguro,The Remains of the Day,2011-08-03,False,3,"{'Fiction': 4784, 'Historical-Historical Ficti..."
47,Aldous Huxley,Brave New World,2011-06-03,False,3,"{'Classics': 17717, 'Fiction': 12564, 'Science..."
48,Kurt Vonnegut Jr.,Slaughterhouse-Five,2011-06-04,False,2,"{'Classics': 12568, 'Fiction': 11930, 'Science..."
49,John Steinbeck,Of Mice and Men,2015-05-11,False,4,"{'Classics': 25191, 'Fiction': 12140, 'Academi..."


Рекомендации


,item_id,score,author,title,genre_and_votes
0,24812,4.918374,Bill Watterson,The Complete Calvin and Hobbes,"{'Sequential Art-Comics': 867, 'Humor': 378, '..."
1,24813,4.851232,Bill Watterson,The Calvin and Hobbes Tenth Anniversary Book,"{'Sequential Art-Comics': 784, 'Humor': 377, '..."
2,22037424,4.828276,"J.K. Rowling, Jonny Duddle, Tomislav Tomić",Harry Potter and the Prisoner of Azkaban (Harr...,"{'Fantasy': 49994, 'Young Adult': 15433, 'Fict..."
3,17332218,4.826075,Brandon Sanderson,"Words of Radiance (The Stormlight Archive, #2)","{'Fantasy': 8542, 'Fiction': 872, 'Fantasy-Epi..."
4,99298,4.822878,"J.K. Rowling, Mary GrandPré","The Harry Potter Collection 1-4 (Harry Potter,...","{'Fantasy': 285, 'Young Adult': 78, 'Fiction':..."
5,30688013,4.817924,Robin Hobb,"Assassin's Fate (The Fitz and the Fool, #3)","{'Fantasy': 1657, 'Fiction': 172, 'Fantasy-Epi..."
6,11295616,4.787009,Ronald Reng,A Life Too Short: The Tragedy of Robert Enke,"{'Sports-Sports': 59, 'Nonfiction': 48, 'Biogr..."
7,32862986,4.786763,Annette Marie,Immortal Fire (Red Winter Trilogy #3),"{'Fantasy': 198, 'Romance': 87, 'Young Adult':..."
8,54741,4.779003,Quino,Toda Mafalda,"{'Sequential Art-Comics': 157, 'Humor': 47, 'S..."
9,747061,4.777556,The Church of Jesus Christ of Latter-day Saints,"Book of Mormon, Doctrine and Covenants, Pearl ...","{'Religion': 249, 'Christianity-Lds': 145, 'No..."


# === Базовые подходы: коллаборативная фильтрация

In [22]:
# Перекодируем идентификаторы пользователей:
# из имеющихся в последовательность 0, 1, 2, ...
user_encoder = sklearn.preprocessing.LabelEncoder()
user_encoder.fit(events["user_id"])
events_train["user_id_enc"] = user_encoder.transform(events_train["user_id"])
events_test["user_id_enc"] = user_encoder.transform(events_test["user_id"])

# Перекодируем идентификаторы объектов:
# из имеющихся в последовательность 0, 1, 2, ...
item_encoder = sklearn.preprocessing.LabelEncoder()
item_encoder.fit(items["item_id"])
items["item_id_enc"] = item_encoder.transform(items["item_id"])
events_train["item_id_enc"] = item_encoder.transform(events_train["item_id"])
events_test["item_id_enc"] = item_encoder.transform(events_test["item_id"])

# Определяем максимальное значение для events_train['item_id_enc']
max_item_id_enc = events_train['item_id_enc'].max()
print(f"Максимальное значение для events_train['item_id_enc']: {max_item_id_enc}")

/tmp/ipykernel_4598/2398154098.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  events_train["user_id_enc"] = user_encoder.transform(events_train["user_id"])
/tmp/ipykernel_4598/2398154098.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  events_test["user_id_enc"] = user_encoder.transform(events_test["user_id"])


Максимальное значение для events_train['item_id_enc']: 43304


/tmp/ipykernel_4598/2398154098.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  events_train["item_id_enc"] = item_encoder.transform(events_train["item_id"])
/tmp/ipykernel_4598/2398154098.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  events_test["item_id_enc"] = item_encoder.transform(events_test["item_id"])


In [24]:
# Вычислим количество уникальных пользователей
num_unique_users = events['user_id'].nunique()

# Вычислим количество уникальных элементов (книг)
num_unique_items = events['item_id'].nunique()

# Размер матрицы в байтах
matrix_size_bytes = num_unique_users * num_unique_items

# Размер матрицы в гигабайтах (целое число)
matrix_size_gb = matrix_size_bytes / (1024 * 1024 * 1024)

print(f"Количество уникальных пользователей: {num_unique_users}")
print(f"Количество уникальных элементов (книг): {num_unique_items}")
print(f"Размер матрицы в байтах: {matrix_size_bytes}")
print(f"Размер матрицы в гигабайтах: {matrix_size_gb}")

Количество уникальных пользователей: 430585
Количество уникальных элементов (книг): 41673
Размер матрицы в байтах: 17943768705
Размер матрицы в гигабайтах: 16.71143686864525


In [25]:
# создаём sparse-матрицу формата CSR 
user_item_matrix_train = scipy.sparse.csr_matrix((
    events_train["rating"],
    (events_train['user_id_enc'], events_train['item_id_enc'])),
    dtype=np.int8)

In [26]:
sum([sys.getsizeof(i) for i in user_item_matrix_train.data])/1024**3 # размер в гигабайтах

0.26370687410235405

In [ ]:
als_model = AlternatingLeastSquares(factors=50, iterations=50, regularization=0.05, random_state=0)
als_model.fit(user_item_matrix_train) 

100%|██████████| 50/50 [02:59<00:00,  3.59s/it]


In [31]:
def get_recommendations_als(user_item_matrix, model, user_id, user_encoder, item_encoder, include_seen=True, n=5):
    """
    Возвращает отранжированные рекомендации для заданного пользователя
    """
    user_id_enc = user_encoder.transform([user_id])[0]
    recommendations = model.recommend(
         user_id_enc, 
         user_item_matrix[user_id_enc], 
         filter_already_liked_items=not include_seen,
         N=n)
    recommendations = pd.DataFrame({"item_id_enc": recommendations[0], "score": recommendations[1]})
    recommendations["item_id"] = item_encoder.inverse_transform(recommendations["item_id_enc"])
    
    return recommendations 

In [32]:
# получаем список всех возможных user_id (перекодированных)
user_ids_encoded = range(len(user_encoder.classes_))

# получаем рекомендации для всех пользователей
als_recommendations = als_model.recommend(
    user_ids_encoded, 
    user_item_matrix_train[user_ids_encoded], 
    filter_already_liked_items=False, N=100) 

In [ ]:
# преобразуем полученные рекомендации в табличный формат
item_ids_enc = als_recommendations[0]
als_scores = als_recommendations[1]

als_recommendations = pd.DataFrame({
    "user_id_enc": user_ids_encoded,
    "item_id_enc": item_ids_enc.tolist(), 
    "score": als_scores.tolist()})
als_recommendations = als_recommendations.explode(["item_id_enc", "score"], ignore_index=True)

# приводим типы данных
als_recommendations["item_id_enc"] = als_recommendations["item_id_enc"].astype("int")
als_recommendations["score"] = als_recommendations["score"].astype("float")

# получаем изначальные идентификаторы
als_recommendations["user_id"] = user_encoder.inverse_transform(als_recommendations["user_id_enc"])
als_recommendations["item_id"] = item_encoder.inverse_transform(als_recommendations["item_id_enc"])
als_recommendations = als_recommendations.drop(columns=["user_id_enc", "item_id_enc"]) 

In [ ]:
als_recommendations = als_recommendations[["user_id", "item_id", "score"]]
als_recommendations.to_parquet("als_recommendations.parquet") 

In [ ]:
als_recommendations = (
    als_recommendations
    .merge(events_test[["user_id", "item_id", "rating"]]
               .rename(columns={"rating": "rating_test"}), 
           on=["user_id", "item_id"], how="left")
) 

# === Базовые подходы: контентные рекомендации

In [ ]:
def compute_ndcg(rating: pd.Series, score: pd.Series, k):

    """ подсчёт ndcg
    rating: истинные оценки
    score: оценки модели
    k: количество айтемов (по убыванию score) для оценки, остальные - отбрасываются
    """
    
    # если кол-во объектов меньше 2, то NDCG - не определена
    if len(rating) < 2:
        return np.nan

    ndcg = sklearn.metrics.ndcg_score(np.asarray([rating.to_numpy()]), np.asarray([score.to_numpy()]), k=k)

    return ndcg 

In [ ]:
rating_test_idx = ~als_recommendations["rating_test"].isnull()
ndcg_at_5_scores = als_recommendations[rating_test_idx].groupby("user_id").apply(lambda x: compute_ndcg(x["rating_test"], x["score"], k=5))

In [ ]:
print(ndcg_at_5_scores.mean()) 

### Факультативное задание

In [ ]:
# Получим список уникальных пользователей в тестовой выборке
unique_users_in_test = events_test['user_id'].unique()

# Получим список пользователей, у которых есть тестовые оценки
users_with_test_ratings = als_recommendations[~als_recommendations["rating_test"].isnull()]['user_id'].unique()

# Количество пользователей с тестовыми оценками
num_users_with_test_ratings = len(users_with_test_ratings)

# Общее количество уникальных пользователей в тестовой выборке
total_unique_users_in_test = len(unique_users_in_test)

# Доля пользователей с посчитанной метрикой NDCG
fraction_with_ndcg = num_users_with_test_ratings / total_unique_users_in_test

print(f"Доля пользователей с посчитанной метрикой NDCG: {fraction_with_ndcg:.2%}")

# 1

In [ ]:
#Составим список всех жанров
items["genre_and_votes"] = items["genre_and_votes"].apply(eval)

In [ ]:
def get_genres(items):
    """
    Извлекает список жанров по всем книгам,
    подсчитывает долю голосов по каждому из них
    """
    genres_counter = {}
    total_votes = 0

    for k, row in items.iterrows():
        genre_and_votes = row['genre_and_votes']
        if genre_and_votes is None or not isinstance(genre_and_votes, dict):
            continue
        for genre, votes in genre_and_votes.items():
            total_votes += votes
            # увеличиваем счетчик жанров
            try:
                genres_counter[genre] += votes
            except KeyError:
                genres_counter[genre] = votes

    # Подсчет долей голосов
    genres = pd.Series(genres_counter, name="votes")
    genres = genres / total_votes if total_votes != 0 else genres  # избегаем деления на ноль
    genres = genres.to_frame()
    genres = genres.reset_index().rename(columns={"index": "name"})
    genres.index.name = "genre_id"

    return genres

genres = get_genres(items)

In [ ]:
genres["score"] = genres["votes"] / genres["votes"].sum()
genres.sort_values(by="score", ascending=False).head(10)

In [ ]:
def get_item2genre_matrix(genres, items):

    genre_names_to_id = genres.reset_index().set_index("name")["genre_id"].to_dict()
    
    # list to build CSR matrix
    genres_csr_data = []
    genres_csr_row_idx = []
    genres_csr_col_idx = []
    
    for item_idx, (k, v) in enumerate(items.iterrows()):
        if v["genre_and_votes"] is None:
            continue
        for genre_name, votes in v["genre_and_votes"].items():
            genre_idx = genre_names_to_id[genre_name]
            genres_csr_data.append(int(votes))
            genres_csr_row_idx.append(item_idx)
            genres_csr_col_idx.append(genre_idx)

    genres_csr = scipy.sparse.csr_matrix((genres_csr_data, (genres_csr_row_idx, genres_csr_col_idx)), shape=(len(items), len(genres)))
    # нормализуем, чтобы сумма оценок принадлежности к жанру была равна 1
    genres_csr = sklearn.preprocessing.normalize(genres_csr, norm='l1', axis=1)
    
    return genres_csr

In [ ]:
# Сортируем товары по item_id_enc
items = items.sort_values(by="item_id_enc")

# Получаем матрицу соответствия товаров и жанров
all_items_genres_csr = get_item2genre_matrix(genres, items)

In [ ]:
# Указываем user_id
user_id = 1000010

# Фильтруем события для данного пользователя и выбираем необходимые столбцы
user_events = events_train.query("user_id == @user_id")[["item_id", "rating"]]

# Отбираем товары, которые были взаимодействованы пользователем
user_items = items[items["item_id"].isin(user_events["item_id"])]

# Получаем матрицу соответствия товаров, взаимодействованных пользователем, и жанров
user_items_genres_csr = get_item2genre_matrix(genres, user_items)
# Выводим полученную матрицу
user_items_genres_csr

In [ ]:
# Выводим количество ненулевых элементов в матрице
non_zero_elements_count = user_items_genres_csr.nnz
non_zero_elements_count

In [ ]:
# вычислим склонность пользователя к жанрам как среднее взвешенное значение популяции на его оценки книг.

# преобразуем пользовательские оценки из списка в вектор-столбец
user_ratings = user_events["rating"].to_numpy() / 5
user_ratings = np.expand_dims(user_ratings, axis=1)

user_items_genres_weighted = user_items_genres_csr.multiply(user_ratings)

user_genres_scores = np.asarray(user_items_genres_weighted.mean(axis=0))

In [ ]:
# выведем список жанров, которые предпочитает пользователь

user_genres = genres.copy()
user_genres["score"] = np.ravel(user_genres_scores)
user_genres = user_genres[user_genres["score"] > 0].sort_values(by=["score"], ascending=False)

user_genres.head(5) 

In [ ]:
# вычисляем сходство между вектором пользователя и векторами по книгам
similarity_scores = cosine_similarity(all_items_genres_csr, user_genres_scores)
# преобразуем в одномерный массив
similarity_scores = similarity_scores.flatten()
# получаем индексы top-k (по убыванию значений), по сути, индексы книг (encoded)
k = 5
# используем np.argsort для получения индексов, сортируя в порядке убывания
top_k_indices = np.argsort(similarity_scores)[-k:]

# Если вы хотите убедиться, что индексы отсортированы по убыванию значений сходства, 
# можно использовать следующий код:
# top_k_indices = np.argsort(similarity_scores)[::-1][:k]

print("Индексы топ-{} книг: {}".format(k, top_k_indices))

In [ ]:
selected_items = items[items["item_id_enc"].isin(top_k_indices)]

with pd.option_context("max_colwidth", 100):
   display(selected_items[["author", "title", "genre_and_votes"]]) 

# === Базовые подходы: валидация

In [ ]:
def process_events_recs_for_binary_metrics(events_train, events_test, recs, top_k=None):

    """
    размечает пары <user_id, item_id> для общего множества пользователей признаками
    - gt (ground truth)
    - pr (prediction)
    top_k: расчёт ведётся только для top k-рекомендаций
    """

    events_test["gt"] = True
    common_users = set(events_test["user_id"]) & set(recs["user_id"])

    print(f"Common users: {len(common_users)}")
    
    events_for_common_users = events_test[events_test["user_id"].isin(common_users)].copy()
    recs_for_common_users = recs[recs["user_id"].isin(common_users)].copy()

    recs_for_common_users = recs_for_common_users.sort_values(["user_id", "score"], ascending=[True, False])

    # оставляет только те item_id, которые были в events_train, 
    # т. к. модель не имела никакой возможности давать рекомендации для новых айтемов
    events_for_common_users = events_for_common_users[events_for_common_users["item_id"].isin(events_train["item_id"].unique())]

    if top_k is not None:
        recs_for_common_users = recs_for_common_users.groupby("user_id").head(top_k)
    
    events_recs_common = events_for_common_users[["user_id", "item_id", "gt"]].merge(
        recs_for_common_users[["user_id", "item_id", "score"]], 
        on=["user_id", "item_id"], how="outer")    

    events_recs_common["gt"] = events_recs_common["gt"].fillna(False)
    events_recs_common["pr"] = ~events_recs_common["score"].isnull()
    
    events_recs_common["tp"] = events_recs_common["gt"] & events_recs_common["pr"]
    events_recs_common["fp"] = ~events_recs_common["gt"] & events_recs_common["pr"]
    events_recs_common["fn"] = events_recs_common["gt"] & ~events_recs_common["pr"]

    return events_recs_common

In [ ]:
events_recs_for_binary_metrics = process_events_recs_for_binary_metrics(
  events_train,
    events_test, 
    als_recommendations, 
    top_k=5)

In [ ]:
def compute_cls_metrics(events_recs_for_binary_metric):
    
    # Группируем данные по user_id
    groupper = events_recs_for_binary_metric.groupby("user_id")
    
    # Вычисляем precision@5 и recall@5 для каждого пользователя
    #precision_5 = groupper.apply(lambda df: df['tp'].head(5).sum() / (df['tp'].head(5).sum() + df['fp'].head(5).sum()) if (df['tp'].head(5).sum() + df['fp'].head(5).sum()) > 0 else 0)
    #ecall_5 = groupper.apply(lambda df: df['tp'].head(5).sum() / (df['tp'].head(5).sum() + df['fn'].head(5).sum()) if (df['tp'].head(5).sum() + df['fn'].head(5).sum()) > 0 else 0)

    # precision = tp / (tp + fp)
    precision_5 = groupper["tp"].sum()/(groupper["tp"].sum()+groupper["fp"].sum())
    precision_5 = precision_5.fillna(0).mean()

    # Вычисляем precision@10 и recall@10 для каждого пользователя
   # precision_10 = groupper.apply(lambda df: df['tp'].head(10).sum() / (df['tp'].head(10).sum() + df['fp'].head(10).sum()) if (df['tp'].head(10).sum() + df['fp'].head(10).sum()) > 0 else 0)
   # recall_10 = groupper.apply(lambda df: df['tp'].head(10).sum() / (df['tp'].head(10).sum() + df['fn'].head(10).sum()) if (df['tp'].head(10).sum() + df['fn'].head(10).sum()) > 0 else 0)
    
    # precision = tp / (tp + fp)
    recall_5 = groupper["tp"].sum()/(groupper["tp"].sum()+groupper["fn"].sum())
    recall_5 = recall_5.fillna(0).mean()
    
    # Вычисляем среднее по всем пользователям
    #precision_5 = precision_5.mean()
    #recall_5 = recall_5.mean()
    
  #  precision_10 = precision_10.mean()
  #  recall_10 = recall_10.mean()

    return precision_5, recall_5 #, precision_10, recall_10

In [ ]:
precision_5, recall_5 = compute_cls_metrics(events_recs_for_binary_metrics)
print(f"Precision@5: {precision_5:.3f}")
print(f"Recall@5: {recall_5:.3f}")


# === Двухстадийный подход: метрики

In [ ]:
als_recommendations = pd.read_parquet("als_recommendations.parquet")

In [ ]:
als_top = als_recommendations.sort_values(['user_id', 'item_id'], ascending=[True, False])

In [ ]:
# Получение всех уникальных item_id
all_items = set(als_top['item_id'].unique())

In [ ]:
# Получение первых 100 строк
first_100_rows = als_top.head(100)

In [ ]:
# Получение уникальных item_id из первых 100 строк
recommended_items = set(first_100_rows['item_id'].unique())

In [ ]:
# Вычисление Item Coverage
item_coverage = len(recommended_items) / len(all_items)

In [ ]:
print(f"Item Coverage для первых 100 строк: {item_coverage:.2f}")

In [ ]:
# Шаг 1: Разметим каждую рекомендацию булевым признаком read, используя данные из events_train
events_train["read"] = True
als_recommendations = als_recommendations.merge(events_train[["user_id", "item_id", "read"]], on=["user_id", "item_id"], how="left")
als_recommendations["read"] = als_recommendations["read"].fillna(False).astype("bool")

In [ ]:
# Шаг 2: Проставим ранги для рекомендаций каждого пользователя
als_recommendations = als_recommendations.sort_values(by=["user_id", "score"], ascending=[True, False])  
# Предполагается, что есть колонка 'score' с оценками рекомендаций
als_recommendations["rank"] = als_recommendations.groupby("user_id").cumcount() + 1

In [ ]:
# Шаг 3: Посчитаем novelty@5 для каждого пользователя
novelty_5 = (1 - als_recommendations.query("rank <= 5").groupby("user_id")["read"].mean())

In [ ]:
# Шаг 4: Посчитаем среднее значение novelty@5
average_novelty_5 = novelty_5.mean()

print(f"Среднее Novelty@5: {average_novelty_5:.2f}")

# === Двухстадийный подход: 

In [ ]:
# задаём точку разбиения
split_date_for_labels = pd.to_datetime("2017-09-15").date()
split_date_for_labels_idx = events_test["started_at"] < split_date_for_labels

In [ ]:
# разделяем данные на две части
events_labels = events_test[split_date_for_labels_idx].copy()
events_test_2 = events_test[~split_date_for_labels_idx].copy()  # используем инвертированную маску для второй части

In [ ]:
events_labels

In [ ]:
# подсчитываем количество уникальных пользователей в events_labels
unique_users_count = events_labels['user_id'].nunique()
print(f"Количество уникальных пользователей в events_labels: {unique_users_count}")

In [ ]:
# загружаем рекомендации от двух базовых генераторов
als_recommendations = pd.read_parquet("candidates/training/als_recommendations.parquet")
content_recommendations = pd.read_parquet("candidates/training/content_recommendations.parquet")

In [ ]:
# объединяем списки рекомендаций по совпадению user_id и item_id
candidates = pd.merge(
    als_recommendations[["user_id", "item_id", "score"]].rename(columns={"score": "als_score"}),
    content_recommendations[["user_id", "item_id", "score"]].rename(columns={"score": "cnt_score"}),
    on=["user_id", "item_id"],  # указываем столбцы для объединения
    how="outer"  # используем внешнее объединение, чтобы сохранить все записи
)

In [ ]:
# выводим количество записей в candidates
print(f"Количество записей в candidates: {candidates.shape[0]}")

In [ ]:
# добавляем таргет к кандидатам со значением:
# — 1 для тех item_id, которые пользователь прочитал
# — 0, для всех остальных 
events_labels["target"] = 1
candidates = candidates.merge(events_labels[["user_id", "item_id", "target"]], 
                              on=["user_id", "item_id"], 
                              how="left")
candidates["target"] = candidates["target"].fillna(0).astype("int")

In [ ]:
# в кандидатах оставляем только тех пользователей, у которых есть хотя бы один положительный таргет
candidates_to_sample = candidates.groupby("user_id").filter(lambda x: x["target"].sum() > 0)

In [ ]:
# для каждого пользователя оставляем только 4 негативных примера
negatives_per_user = 4
candidates_for_train = pd.concat([
    candidates_to_sample.query("target == 1"),  # положительные примеры
    candidates_to_sample.query("target == 0") \
        .groupby("user_id") \
        .apply(lambda x: x.sample(negatives_per_user, random_state=0))
    ])

In [ ]:
# Сверим часы: сколько записей получилось в candidates_for_train?
print(f"Количество записей в candidates_for_train: {len(candidates_for_train)}")

# === Двухстадийный подход: модель

In [ ]:
# задаём имена колонок признаков и таргета
features = ['als_score', 'cnt_score']
target = 'target'

In [ ]:
# Create the Pool object
train_data = Pool(
    data=candidates_for_train[features], 
    label=candidates_for_train[target])

In [ ]:
# инициализируем модель CatBoostClassifier
cb_model = CatBoostClassifier(
    iterations=1000,
    learning_rate=0.1,
    depth=6,
    loss_function='Logloss',
    verbose=100,
    random_seed=0
)

In [ ]:
# тренируем модель
cb_model.fit(train_data) 

# === Двухстадийный подход: построение признаков

In [ ]:
# загружаем рекомендации от двух базовых генераторов
als_recommendations_2 = pd.read_parquet("candidates/inference/als_recommendations.parquet")
content_recommendations_2 = pd.read_parquet("candidates/inference/content_recommendations.parquet")

In [ ]:
# объединяем кандидатов от обоих базовых генераторов
candidates_to_rank = pd.merge(
    als_recommendations_2[["user_id", "item_id", "score"]].rename(columns={"score": "als_score"}),
    content_recommendations_2[["user_id", "item_id", "score"]].rename(columns={"score": "cnt_score"}),
    on=["user_id", "item_id"],  # указываем столбцы для объединения
    how="outer"  # используем внешнее объединение, чтобы сохранить все записи
)

In [ ]:
# оставляем только тех пользователей, что есть в тестовой выборке, для экономии ресурсов
candidates_to_rank = candidates_to_rank[candidates_to_rank["user_id"].isin(events_test_2["user_id"].drop_duplicates())]

In [ ]:
# выводим количество записей в candidates_to_rank
print(len(candidates_to_rank))

In [ ]:
inference_data = Pool(data=candidates_to_rank[features])

In [ ]:
predictions = cb_model.predict_proba(inference_data)

In [ ]:
candidates_to_rank["cb_score"] = predictions[:, 1]

In [ ]:
# Сортируем данные по пользователю и по убыванию cb_score внутри каждого пользователя
candidates_to_rank = candidates_to_rank.sort_values(["user_id", "cb_score"], ascending=[True, False])

In [ ]:
# Присваиваем ранги внутри каждой группы пользователей
candidates_to_rank["rank"] = candidates_to_rank.groupby("user_id").cumcount() + 1

In [ ]:
max_recommendations_per_user = 100

In [ ]:
# Фильтруем данные, оставляя только те строки, где rank <= max_recommendations_per_user
final_recommendations = candidates_to_rank[candidates_to_rank["rank"] <= max_recommendations_per_user]

In [ ]:
# Если вы хотите убедиться, что у каждого пользователя ровно max_recommendations_per_user рекомендаций,
# можно использовать следующий подход:
#final_recommendations = candidates_to_rank.groupby('user_id').head(max_recommendations_per_user)

In [ ]:
# Проверяем количество записей в final_recommendations
print(final_recommendations.shape[0])

In [ ]:
events_inference = pd.concat([events_train, events_labels])

In [ ]:
# Переименовываем столбец с оценками рекомендаций для удобства
final_recommendations_renamed = final_recommendations.rename(columns={"cb_score": "score"})

# Обрабатываем события и рекомендации для расчета бинарных метрик
cb_events_recs_for_binary_metrics_5 = process_events_recs_for_binary_metrics(
    events_inference,  # DataFrame с тренировочными событиями
    events_test_2,  # DataFrame с тестовыми событиями
    final_recommendations_renamed,  # Переименованный DataFrame с рекомендациями
    top_k=5  # Количество топ-рекомендаций для оценки
)

# Рассчитываем метрики precision и recall
precision_5, recall_5 = compute_cls_metrics(cb_events_recs_for_binary_metrics_5)

# Выводим результаты
print(f"precision@5: {precision_5:.3f}, recall@5: {recall_5:.3f}")
#print(f"precision@10: {precision_10:.3f}, recall@10: {recall_10:.3f}")

In [ ]:
# Добавляем признак возраста age к items
items["age"] = 2018 - items["publication_year"]
invalid_age_idx = items["age"] < 0
items.loc[invalid_age_idx, "age"] = np.nan
items["age"] = items["age"].astype("float")

# Добавляем признаки age и average_rating к кандидатам для обучения
candidates_for_train = candidates_for_train.merge(items[["item_id", "age", "average_rating"]], 
                                                 on="item_id", how="left")

# Добавляем признаки age и average_rating к кандидатам для ранжирования
candidates_to_rank = candidates_to_rank.merge(items[["item_id", "age", "average_rating"]], 
                                               on="item_id", how="left")

In [ ]:
# Вычисляем медианный возраст книг в candidates_to_rank
median_age = candidates_to_rank['age'].median()

# Выводим результат
print(f"Медианный возраст книги в candidates_to_rank: {median_age}")

In [ ]:
def get_user_features(events):
    """ Считает пользовательские признаки """
    user_features = events.groupby("user_id").agg(
        reading_years=("started_at", lambda x: (x.max() - x.min()).days / 365.25),
        books_read=("item_id", "nunique"),  # считаем уникальные книги, прочитанные пользователем
        rating_avg=("rating", "mean"),
        rating_std=("rating", "std")
    )
    user_features["books_per_year"] = user_features["books_read"] / user_features["reading_years"]
    return user_features

In [ ]:
# user_features_for_train уже рассчитано в предыдущем шаге
user_features_for_train = get_user_features(events_train)

# Добавляем признаки пользователей к кандидатам
candidates_for_train = candidates_for_train.merge(user_features_for_train, on="user_id", how="left")

# Вычисляем медианное значение количества прочитанных книг
median_books_read = candidates_for_train["books_read"].median()

print(f"Медиана количества прочитанных книг по всем кандидатам в candidates_for_train: {median_books_read}")

In [ ]:
# Получаем признаки для ранжирования
user_features_for_ranking = get_user_features(events_inference)

# Добавляем эти признаки к набору кандидатов для ранжирования
candidates_to_rank = candidates_to_rank.merge(user_features_for_ranking, on="user_id", how="left")

In [ ]:
genres_top_k = 10
genres_top_idx = genres.sort_values("votes", ascending=False).head(genres_top_k).index
genres_others_idx = list(set(genres.index) - set(genres_top_idx))
genres_top_columns = [f"genre_{id}" for id in genres_top_idx]
genres_others_column = "genre_others"
genre_columns = genres_top_columns + [genres_others_column]  # объединяем столбцы топ-жанров и others

# составляем таблицу принадлежности книг к жанрам
item_genres = (
    pd.concat([
        # топ жанров
        pd.DataFrame(all_items_genres_csr[:, genres_top_idx].toarray(), columns=genres_top_columns),
        # все остальные жанры
        pd.DataFrame(all_items_genres_csr[:, genres_others_idx].sum(axis=1), columns=[genres_others_column])
    ],
    axis=1)
    .reset_index()
    .rename(columns={"index": "item_id_enc"})
)

# объединяем информацию принадлежности книг к жанрам с основной информацией о книгах
items = items.merge(item_genres, on="item_id_enc", how="left")

def get_user_genres(events, items, item_genre_columns):
    user_genres = (
        events
        .merge(items[["item_id"] + item_genre_columns], on="item_id", how="left")
        .groupby("user_id")[item_genre_columns].mean()
    )
    return user_genres

user_genres_for_train = get_user_genres(events_train, items, genre_columns)  # расчёт для тренировочных данных
candidates_for_train = candidates_for_train.merge(user_genres_for_train, on="user_id", how="left")

user_genres_for_ranking = get_user_genres(events_inference, items, genre_columns)  # расчёт для ранжирования
candidates_to_rank = candidates_to_rank.merge(user_genres_for_ranking, on="user_id", how="left")

In [ ]:
# Сортируем жанры по количеству голосов в порядке убывания
sorted_genres = genres.sort_values("votes", ascending=False)

# Находим индекс жанра "Romance" в отсортированном списке
romance_rank = sorted_genres[sorted_genres['name'] == "Romance"].index[0] + 1  # +1, чтобы получить порядковый номер, а не индекс

print(f"Жанр 'Romance' занимает {romance_rank}-е место в списке популярных жанров.")

In [ ]:
median_romance = candidates_for_train['genre_34'].median()
print(f"Медиана жанровости для жанра 'Romance': {median_romance:.2f}")

In [ ]:
# задаём имена колонок признаков и таргета
features = ['als_score', 'cnt_score', 
    'age', 'average_rating', 'reading_years', 'books_read', 
    'rating_avg', 'rating_std', 
    'books_per_year'] + genre_columns
target = 'target'

# создаём Pool
train_data = Pool(
    data=candidates_for_train[features], 
    label=candidates_for_train[target])

# инициализируем модель CatBoostClassifier
cb_model = CatBoostClassifier(
    iterations=1000,
    learning_rate=0.1,
    depth=6,
    loss_function='Logloss',
    verbose=100,
    random_seed=0,
)

# тренируем модель
cb_model.fit(train_data)

In [ ]:
# создаем Pool для инференса
inference_data = Pool(data=candidates_to_rank[features])

# предсказываем вероятности классов (1 - положительный класс, 0 - отрицательный класс)
predictions = cb_model.predict_proba(inference_data)

# сохраняем вероятности положительного класса (второй столбец)
candidates_to_rank["cb_score"] = predictions[:, 1]

# сортируем кандидатов по пользователю и по убыванию cb_score
candidates_to_rank = candidates_to_rank.sort_values(["user_id", "cb_score"], ascending=[True, False])

# проставляем ранк, начиная с 1 для каждого пользователя
candidates_to_rank["rank"] = candidates_to_rank.groupby("user_id").cumcount() + 1

# фильтруем кандидатов, оставляя только топ-100 для каждого пользователя
max_recommendations_per_user = 100
final_recommendations = candidates_to_rank.query("rank <= @max_recommendations_per_user")

In [ ]:
candidates_to_rank

In [ ]:
# Подсчитываем количество уникальных пользователей в final_recommendations
unique_users_count = final_recommendations['user_id'].nunique()
print(f"Количество уникальных пользователей в final_recommendations: {unique_users_count}")

In [ ]:
# Сохраняем final_recommendations в файл final_recommendations_feat.parquet
final_recommendations.to_parquet('final_recommendations_feat.parquet')

In [ ]:
# для экономии ресурсов оставим события только тех пользователей, 
# для которых следует оценить рекомендации
events_inference = pd.concat([events_train, events_labels])
events_inference = events_inference[events_inference["user_id"].isin(events_test_2["user_id"].drop_duplicates())]

In [ ]:
# Обработка событий и рекомендаций
cb_events_recs_for_binary_metrics_5 = process_events_recs_for_binary_metrics(
    events_inference,
    events_labels,
    final_recommendations.rename(columns={"cb_score": "score"}),
    5
)

In [ ]:
# Вычисление метрик precision и recall
cb_precision_5, cb_recall_5 = compute_cls_metrics(cb_events_recs_for_binary_metrics_5)

In [ ]:
print(f"precision: {cb_precision_5:.3f}, recall: {cb_recall_5:.3f}")

In [ ]:
# Получаем важность признаков
feature_importance = pd.DataFrame(cb_model.get_feature_importance(), 
                                  index=features, 
                                  columns=["fi"])

# Сортируем признаки по убыванию их важности
feature_importance = feature_importance.sort_values(by="fi", ascending=False)

# Выводим отсортированный DataFrame с важностью признаков
print(feature_importance)

In [30]:
# получим энкодированные идентификаторы всех объектов, известных нам из events_train
train_item_ids_enc = events_train['item_id_enc'].unique()
max_similar_items = 10
# получаем списки похожих объектов, используя ранее полученную ALS-модель
# метод similar_items возвращает и сам объект, как наиболее похожий
# этот объект мы позже отфильтруем, но сейчас запросим на 1 больше
similar_items = als_model.similar_items(train_item_ids_enc, N=max_similar_items+1)

# Преобразуем в список словарей
items_list = []
for item_id_enc, sim_items, scores in zip(train_item_ids_enc, similar_items[0], similar_items[1]):
    for sim_item, score in zip(sim_items, scores):
        items_list.append({
            "item_id_enc": item_id_enc,
            "sim_item_id_enc": sim_item,
            "score": score
        })

# Создаем DataFrame из списка словарей
similar_items_df = pd.DataFrame(items_list)

# Разворачиваем списки в отдельные строки
# (это шаг, который не требуется здесь, так как мы уже создали DataFrame с развернутыми данными)
# similar_items = similar_items.explode(["sim_item_id_enc", "score"])

# Приводим типы данных
similar_items_df["sim_item_id_enc"] = similar_items_df["sim_item_id_enc"].astype("int")
similar_items_df["score"] = similar_items_df["score"].astype("float")

# Получаем изначальные идентификаторы
similar_items_df["item_id_1"] = similar_items_df["item_id_enc"].map(item_id_encoder.inverse_transform)
similar_items_df["item_id_2"] = similar_items_df["sim_item_id_enc"].map(item_id_encoder.inverse_transform)

# Убираем пары с одинаковыми объектами
similar_items_df = similar_items_df.query("item_id_1 != item_id_2")


NameError: name 'item_id_encoder' is not defined